In [3]:
from ipyleaflet import *

m = Map(center=(52,10), zoom=8, basemap = basemaps.Esri.DeLorme)
m

Map(center=[52, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…